## Import & Defines

In [ ]:
import time
import os
from IPython import display
import pandas as pd
import requests

# Convert a time in seconds in a string of type hh:mm:ss
def hmsString(seconds) : 
    h = int(seconds / (60 * 60))
    m = int((seconds % (60 * 60)) / 60)
    s = seconds % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

import string
import xml
import re
import sys

In [2]:
dfRecipePathToUse = "../difford/recipes.pkl"
dfIngredientsPathToUse = "../difford/ingredients.pkl"

dfRecipes = pd.read_pickle(dfRecipePathToUse)
dfIngredients = pd.read_pickle(dfIngredientsPathToUse)

## Convert to Markdown

In [526]:
def dataframeToMarkdown(df, source) :
    markdownPath = f"cocktails/{source}/markdown.md"
    printEvery = 100
    
    totalItems = len(df.index)
    totalItemsString = "{:,}".format(totalItems).replace(",", " ")
    itemCount = 0
    startTime = time.time()
    
    f = open(markdownPath, mode="w", encoding="utf8")
    f.write(f"# Cocktails from {source}\n\n")
    for index, row in df.iterrows() :
        f.write(f"## {row['name']}\n\n")
        #f.write(f"<img style='float: right;' width='20%' src='{row['picture']}'>\n")
        f.write(f"<img style='float: right;' width='20%' src='./../../{row['downloadedPicture']}'>\n")
        
        f.write(f"\n### Information \n\n")
        f.write(f" - Glass: {row['glass']}\n")
        f.write(f" - Garnish: {row['garnish']}\n")
        f.write(f" - How to make: {row['instructions']}\n")
        
        
        f.write(f"\n### Ingredients \n\n")
        
        ingredients = row["ingredients"]
        for ingr in ingredients :
            qty = ingredients[ingr]["quantity"]
            unit = ingredients[ingr]["unit"]
            f.write(f" - {qty} {unit} of {ingr}\n")
                    
        f.write("\n")
        
        itemCount += 1 

        if (itemCount > 0 and itemCount % printEvery == 0) or itemCount == totalItems :
            countItemsString = "{:,}".format(itemCount).replace(",", " ")
            percentageString = "{:.4}%".format(100.0 * itemCount / totalItems)
            timeElapsed = time.time() - startTime
            eta = int((totalItems - itemCount) * timeElapsed / itemCount)
            string = (f"{countItemsString} / {totalItemsString} done ({percentageString})" + 
                f" > {hmsString(timeElapsed)} : {hmsString(eta)} ({hmsString(timeElapsed + eta)})")
            display.clear_output(wait=True)
            print(string)

In [527]:
dataframeToMarkdown(dfRecipes, "difford")

4 812 / 4 812 done (100.0%) > 0:00:01.24 : 0:00:00.00 (0:00:01.24)


## Convert to LaTeX

Direct Writing to File

In [176]:
import re

def writeInLatexFile(file, string) :
    thingToWrite = string.replace("&", "\&").replace("%", "\%").replace("#", "\#")
    thingToWrite = thingToWrite.replace("˚", "\\textdegree ").replace("°", "\\textdegree ")
    thingToWrite = thingToWrite.replace("⅐", "1/7").replace("⅑", "1/9").replace("⅒", "1/10")
    thingToWrite = thingToWrite.replace("⅓", "1/3").replace("⅔", "2/3").replace("⅕", "1/5")
    thingToWrite = thingToWrite.replace("⅖", "2/5").replace("⅗", "3/5").replace("⅘", "4/5")
    thingToWrite = thingToWrite.replace("⅙", "1/6").replace("⅚", "5/6").replace("⅛", "1/8")
    thingToWrite = thingToWrite.replace("⅜", "3/8").replace("⅝", "5/8").replace("⅞", "7/8")
    thingToWrite = re.sub(r"([0-9]+)/([0-9]+)", r"\\nicefrac{\1}{\2}", thingToWrite)
    file.write(thingToWrite)

def writeRating(rating, total) :
    fullStar = "\\faStar "
    halfFullStar = "\\faStarHalfO "
    emptyStar = "\\faStarO "
    ratingString = ""
    rating *= total
    for n in range(total) :
        if rating >= 1 :
            ratingString += fullStar
        elif rating > 0 :
            ratingString += halfFullStar
        else :
            ratingString += emptyStar
        rating -= 1
    return ratingString
    
def dataframeToLatex(df, source, sourceName=None) :
    if sourceName == None :
        sourceName = source
        
    df = df.sort_values(by="name")
    markdownPath = f"cocktails/{source}/latex/{source}.tex"
    printEvery = 100
    
    totalItems = len(df.index)
    totalItemsString = "{:,}".format(totalItems).replace(",", " ")
    itemCount = 0
    startTime = time.time()
    
    f = open(markdownPath, mode="w", encoding="utf8")

    writeInLatexFile(f, "\\documentclass[8pt, a4paper, twocolumn]{article}\n")
    writeInLatexFile(f, "\\usepackage{graphicx}\n")
    writeInLatexFile(f, "\\usepackage{wrapfig}\n")
    writeInLatexFile(f, "\\usepackage{nicefrac}\n")
    writeInLatexFile(f, "\\usepackage{fontawesome}\n")
    writeInLatexFile(f, "\\begin{document}\n\n")
    writeInLatexFile(f, "    \\graphicspath{\n")
    writeInLatexFile(f, "        {../pictures/}\n")
    writeInLatexFile(f, "    }\n\n")
    writeInLatexFile(f, "    \\title{Cocktails from \\textit{?}}\n\n".replace("?", sourceName))
    writeInLatexFile(f, "    \\maketitle\n\n")
    for index, row in df.iterrows() :
        writeInLatexFile(f, "    \\normalsize\n")
        writeInLatexFile(f, "    \\section{?}\n\n".replace("?", row['name']))
        writeInLatexFile(f, "            \\subsection{Information} \n\n")
        #f.write(f"<img style='float: right;' width='20%' src='{row['picture']}'>\n")
        
        if row["downloadedPicture"] != None :
            
            writeInLatexFile(f, "      \\begin{minipage}{.35\\textwidth}\n")            
        
            pictureName = row['downloadedPicture'].split("/")[-1].split(".")[0]
        
            writeInLatexFile(f, "                \\begin{itemize}\n")
            rating = row["rating"]["publisher"]["rating"]
            if rating != None :
                writeInLatexFile(f, "                    \\item [Rating]: ?\n".replace("?", writeRating(rating, 5)))
            writeInLatexFile(f, "                    \\item [Glass]: ?\n".replace("?", row['glass']))
            writeInLatexFile(f, "                    \\item [Garnish]: ?\n".replace("?", row['garnish']))
            writeInLatexFile(f, "                    \\item [Make]: ?\n".replace("?", row['instructions']))
            writeInLatexFile(f, "                \\end{itemize}\n")

            writeInLatexFile(f, "           \\end{minipage}\n")

            writeInLatexFile(f, "           \\begin{minipage}{.1\\textwidth}\n")

            writeInLatexFile(f, "            \\includegraphics[width=\\textwidth]{?}\n".replace("?", pictureName))

            writeInLatexFile(f, "           \\end{minipage}\n\n")
            
        else :
            writeInLatexFile(f, "                \\begin{itemize}\n")
            rating = row["rating"]["publisher"]["rating"]
            if rating != None :
                writeInLatexFile(f, "                    \\item [Rating]: ?\n".replace("?", writeRating(rating, 5)))
            writeInLatexFile(f, "                    \\item [Glass]: ?\n".replace("?", row['glass']))
            writeInLatexFile(f, "                    \\item [Garnish]: ?\n".replace("?", row['garnish']))
            writeInLatexFile(f, "                    \\item [How to make]: ?\n".replace("?", row['instructions']))
            writeInLatexFile(f, "                \\end{itemize}\n")

        writeInLatexFile(f, "        \\subsection{Ingredients} \n\n")
        writeInLatexFile(f, "            \\begin{itemize}\n")
        
        ingredients = row["ingredients"]
        for ingr in ingredients :
            qty = ingredients[ingr]["quantity"]
            unit = ingredients[ingr]["unit"]
            writeInLatexFile(f, "                \\item {?} {§} of {!}\n".replace("?", qty).replace("§", unit).replace("!", ingr))
            
        writeInLatexFile(f, "            \\end{itemize}\n\n")
        
        itemCount += 1 

        if (itemCount > 0 and itemCount % printEvery == 0) or itemCount == totalItems :
            countItemsString = "{:,}".format(itemCount).replace(",", " ")
            percentageString = "{:.4}%".format(100.0 * itemCount / totalItems)
            timeElapsed = time.time() - startTime
            eta = int((totalItems - itemCount) * timeElapsed / itemCount)
            string = (f"{countItemsString} / {totalItemsString} done ({percentageString})" + 
                f" > {hmsString(timeElapsed)} : {hmsString(eta)} ({hmsString(timeElapsed + eta)})")
            display.clear_output(wait=True)
            print(string)
    
        
    writeInLatexFile(f, "    \\setcounter{tocdepth}{1}\n")

    writeInLatexFile(f, "    \\tableofcontents\n")
    writeInLatexFile(f, "\n")

    writeInLatexFile(f, "\end{document}\n")
    
    f.close()

Using String before writing to File

In [1]:
import re

def rowToLatexString(index, row) :
    resultString = ""
    
    resultString += getLatexReadyString("    \\normalsize\n")
    resultString += getLatexReadyString("    \\section{?}\n\n".replace("?", row['name']))
    resultString += getLatexReadyString("            \\subsection{Information} \n\n")
    #f.write(f"<img style='float: right;' width='20%' src='{row['picture']}'>\n")

    if row["downloadedPicture"] != None :

        resultString += getLatexReadyString("      \\begin{minipage}{.35\\textwidth}\n")            

        pictureName = row['downloadedPicture'].split("/")[-1].split(".")[0]

        resultString += getLatexReadyString("                \\begin{itemize}\n")
        rating = row["rating"]["publisher"]["rating"]
        if rating != None :
            resultString += getLatexReadyString("                    \\item [Rating]: ?\n".replace("?", writeRating(rating, 5)))
        resultString += getLatexReadyString("                    \\item [Glass]: ?\n".replace("?", row['glass']))
        resultString += getLatexReadyString("                    \\item [Garnish]: ?\n".replace("?", row['garnish']))
        resultString += getLatexReadyString("                    \\item [Make]: ?\n".replace("?", row['instructions']))
        resultString += getLatexReadyString("                \\end{itemize}\n")

        resultString += getLatexReadyString("           \\end{minipage}\n")

        resultString += getLatexReadyString("           \\begin{minipage}{.1\\textwidth}\n")

        resultString += getLatexReadyString("            \\includegraphics[width=\\textwidth]{?}\n".replace("?", pictureName))

        resultString += getLatexReadyString("           \\end{minipage}\n\n")

    else :
        resultString += getLatexReadyString("                \\begin{itemize}\n")
        rating = row["rating"]["publisher"]["rating"]
        if rating != None :
            resultString += getLatexReadyString("                    \\item [Rating]: ?\n".replace("?", writeRating(rating, 5)))
        resultString += getLatexReadyString("                    \\item [Glass]: ?\n".replace("?", row['glass']))
        resultString += getLatexReadyString("                    \\item [Garnish]: ?\n".replace("?", row['garnish']))
        resultString += getLatexReadyString("                    \\item [How to make]: ?\n".replace("?", row['instructions']))
        resultString += getLatexReadyString("                \\end{itemize}\n")

    resultString += getLatexReadyString("        \\subsection{Ingredients} \n\n")
    resultString += getLatexReadyString("            \\begin{itemize}\n")

    ingredients = row["ingredients"]
    for ingr in ingredients :
        qty = ingredients[ingr]["quantity"]
        unit = ingredients[ingr]["unit"]
        resultString += getLatexReadyString("                \\item {?} {§} of {!}\n".replace("?", qty).replace("§", unit).replace("!", ingr))

    resultString += getLatexReadyString("            \\end{itemize}\n\n")
    
    return resultString

def getLatexReadyString(string) :
    thingToWrite = string.replace("&", "\&").replace("%", "\%").replace("#", "\#")
    thingToWrite = thingToWrite.replace("˚", "\\textdegree ").replace("°", "\\textdegree ")
    thingToWrite = thingToWrite.replace("⅐", "1/7").replace("⅑", "1/9").replace("⅒", "1/10")
    thingToWrite = thingToWrite.replace("⅓", "1/3").replace("⅔", "2/3").replace("⅕", "1/5")
    thingToWrite = thingToWrite.replace("⅖", "2/5").replace("⅗", "3/5").replace("⅘", "4/5")
    thingToWrite = thingToWrite.replace("⅙", "1/6").replace("⅚", "5/6").replace("⅛", "1/8")
    thingToWrite = thingToWrite.replace("⅜", "3/8").replace("⅝", "5/8").replace("⅞", "7/8")
    thingToWrite = re.sub(r"([0-9]+)/([0-9]+)", r"\\nicefrac{\1}{\2}", thingToWrite)
    return thingToWrite

def writeInLatexFile(file, string) :
    thingToWrite = getLatexReadyString(string)
    file.write(thingToWrite)

def writeRating(rating, total) :
    fullStar = "\\faStar "
    halfFullStar = "\\faStarHalfO "
    emptyStar = "\\faStarO "
    ratingString = ""
    rating *= total
    for n in range(total) :
        if rating >= 1 :
            ratingString += fullStar
        elif rating > 0 :
            ratingString += halfFullStar
        else :
            ratingString += emptyStar
        rating -= 1
    return ratingString
    
def dataframeToLatex(df, source, sourceName=None) :
    if sourceName == None :
        sourceName = source
        
    df = df.sort_values(by="name")
    markdownPath = f"cocktails/{source}/latex/{source}.tex"
    printEvery = 100
    
    totalItems = len(df.index)
    totalItemsString = "{:,}".format(totalItems).replace(",", " ")
    itemCount = 0
    startTime = time.time()
    
    f = open(markdownPath, mode="w", encoding="utf8")

    writeInLatexFile(f, "\\documentclass[8pt, a4paper, twocolumn]{article}\n")
    writeInLatexFile(f, "\\usepackage{graphicx}\n")
    writeInLatexFile(f, "\\usepackage{wrapfig}\n")
    writeInLatexFile(f, "\\usepackage{nicefrac}\n")
    writeInLatexFile(f, "\\usepackage{fontawesome}\n")
    writeInLatexFile(f, "\\begin{document}\n\n")
    writeInLatexFile(f, "    \\graphicspath{\n")
    writeInLatexFile(f, "        {../pictures/}\n")
    writeInLatexFile(f, "    }\n\n")
    writeInLatexFile(f, "    \\title{Cocktails from \\textit{?}}\n\n".replace("?", sourceName))
    writeInLatexFile(f, "    \\maketitle\n\n")
    for index, row in df.iterrows() :
        
        thingToWrite = rowToLatexString(index, row)
        file.write(thingToWrite)
        
        itemCount += 1 

        if (itemCount > 0 and itemCount % printEvery == 0) or itemCount == totalItems :
            countItemsString = "{:,}".format(itemCount).replace(",", " ")
            percentageString = "{:.4}%".format(100.0 * itemCount / totalItems)
            timeElapsed = time.time() - startTime
            eta = int((totalItems - itemCount) * timeElapsed / itemCount)
            string = (f"{countItemsString} / {totalItemsString} done ({percentageString})" + 
                f" > {hmsString(timeElapsed)} : {hmsString(eta)} ({hmsString(timeElapsed + eta)})")
            display.clear_output(wait=True)
            print(string)
    
        
    writeInLatexFile(f, "    \\setcounter{tocdepth}{1}\n")

    writeInLatexFile(f, "    \\tableofcontents\n")
    writeInLatexFile(f, "\n")

    writeInLatexFile(f, "\end{document}\n")
    
    f.close()

In [626]:
sample = dfRecipes.sample(10)

In [177]:
dataframeToLatex(dfRecipes, "difford", "Difford's Guide")

4 802 / 4 802 done (100.0%) > 0:00:02.63 : 0:00:00.00 (0:00:02.63)
